## Analysis

In [1]:
from time import time
import math, os
import numpy as np
from sklearn import metrics
import h5py
import scanpy as sc
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import sys
sys.path.append('./Baseline_Code')
sys.path.append('./EnhancedscDeepCluster')

In [2]:
# scDeepCluster
from scDeepCluster import scDeepCluster
from single_cell_tools import *
from preprocess import read_dataset, normalize

In [3]:
# ModifiedscDeepCluster
import model
import model_single_cell_tools
import model_preprocess

In [4]:
# for repeatability
torch.manual_seed(42)

Setup parameters.

In [5]:
'''
Parameter setting
'''

class Args(object):
    def __init__(self):
        self.n_clusters = 8
        self.knn = 20
        self.resolution = 0.8
        self.select_genes = 0
        self.batch_size = 256
        self.data_file = '10X_PBMC.h5'
        self.maxiter = 2000
        self.pretrain_epochs = 300
        self.gamma = 1.
        self.sigma = 2.5
        self.update_interval = 1
        self.tol = 0.001
        self.use_attention = True
        self.use_layernorm = True
        self.use_batchnorm = False
        # self.ae_weights = None
        self.ae_weights = '10xPBMC_with_attn_plus_layer_weights.pth.tar'
        self.save_dir = 'results/EnhancedscDeepCluster/'
        self.ae_weight_file = '10xPBMC_with_attn_plus_layer_weights.pth.tar'
        self.final_latent_file = '10xPBMC_final_latent_file.txt'
        self.predict_label_file = '10xPBMC_pred_labels.txt'
        self.device = 'cuda'

args = Args()

Normalizating and preprocessing count data.

In [6]:
data_mat = h5py.File(args.data_file)
x = np.array(data_mat['X'])
# y is the ground truth labels for evaluating clustering performance
# If not existing, we skip calculating the clustering performance metrics (e.g. NMI ARI)
if 'Y' in data_mat:
    y = np.array(data_mat['Y'])
else:
    y = None
data_mat.close()

if args.select_genes > 0:
    importantGenes = model_single_cell_tools.geneSelection(x, n=args.select_genes, plot=False)
    x = x[:, importantGenes]

# preprocessing scRNA-seq read counts matrix
adata = sc.AnnData(x)
if y is not None:
    adata.obs['Group'] = y

adata = model_preprocess.read_dataset(adata,
                 transpose=False,
                 test_split=False,
                 copy=True)

adata = model_preprocess.normalize(adata,
                  size_factors=True,
                  normalize_input=True,
                  logtrans_input=True)

input_size = adata.n_vars

print(args)

print(adata.X.shape)
if y is not None:
    print(y.shape)

### Autoencoder: Successfully preprocessed 16653 genes and 4271 cells.
(4271, 16653)
(4271,)


Build scDeepCluster model

In [7]:
model = model.EnhancedscDeepCluster(input_dim=adata.n_vars, z_dim=32, use_attention=args.use_attention, use_layernorm=args.use_layernorm, use_batchnorm=args.use_batchnorm,
            encodeLayer=[256, 64], decodeLayer=[64, 256], sigma=args.sigma, gamma=args.gamma, device=args.device)

print(str(model))

C:\Users\bhari\Downloads\EnhancedscDeepCluster\project\lib\site-packages\torch\nn\modules\transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


EnhancedscDeepCluster(
  (dropoutLayer): Dropout(p=0.8, inplace=False)
  (encoder): Sequential(
    (0): Linear(in_features=16653, out_features=256, bias=True)
    (1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (2): ReLU()
    (3): Linear(in_features=256, out_features=64, bias=True)
    (4): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (5): ReLU()
  )
  (attention): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
        )
        (linear1): Linear(in_features=64, out_features=64, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=64, out_features=64, bias=True)
        (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=

Pretraining stage.

In [8]:
t0 = time()
if args.ae_weights is None:
    model.pretrain_autoencoder(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, 
                            batch_size=args.batch_size, epochs=args.pretrain_epochs, ae_weights=args.ae_weight_file)
else:
    if os.path.isfile(args.ae_weights):
        print("==> loading checkpoint '{}'".format(args.ae_weights))
        checkpoint = torch.load(args.ae_weights)
        model.load_state_dict(checkpoint['ae_state_dict'])
    else:
        print("==> no checkpoint found at '{}'".format(args.ae_weights))
        raise ValueError

print('Pretraining time: %d seconds.' % int(time() - t0))

==> loading checkpoint '10xPBMC_with_attn_plus_layer_weights.pth.tar'
Pretraining time: 0 seconds.


Pretrain epoch 196, ZINB loss: 0.22281503
Pretrain epoch 197, ZINB loss: 0.22263191
Pretrain epoch 198, ZINB loss: 0.22256174
Pretrain epoch 199, ZINB loss: 0.22243141
Pretrain epoch 200, ZINB loss: 0.22252310
Pretrain epoch 201, ZINB loss: 0.22236248
Pretrain epoch 202, ZINB loss: 0.22244692
Pretrain epoch 203, ZINB loss: 0.22217898
Pretrain epoch 204, ZINB loss: 0.22209356
Pretrain epoch 205, ZINB loss: 0.22210407
Pretrain epoch 206, ZINB loss: 0.22195778
Pretrain epoch 207, ZINB loss: 0.22193105
Pretrain epoch 208, ZINB loss: 0.22195967
Pretrain epoch 209, ZINB loss: 0.22182915
Pretrain epoch 210, ZINB loss: 0.22169341
Pretrain epoch 211, ZINB loss: 0.22163370
Pretrain epoch 212, ZINB loss: 0.22177499
Pretrain epoch 213, ZINB loss: 0.22147643
Pretrain epoch 214, ZINB loss: 0.22148880
Pretrain epoch 215, ZINB loss: 0.22136244
Pretrain epoch 216, ZINB loss: 0.22121936
Pretrain epoch 217, ZINB loss: 0.22130410
Pretrain epoch 218, ZINB loss: 0.22126952
Pretrain epoch 219, ZINB loss: 0.2

Clustering stage.

In [9]:
if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

if args.n_clusters > 0:
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X.astype(np.float32), size_factor=adata.obs.size_factors, n_clusters=args.n_clusters, init_centroid=None, 
                y_pred_init=None, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)
else:
    cluster_centers, n_clusters, y_pred_init = model_preprocess.louvain_init_clustering(model=model, adata=adata, knn=args.knn, resolution=args.resolution)
    print('Estimated number of clusters: ', n_clusters)
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, n_clusters=n_clusters, init_centroid=cluster_centers, 
                y_pred_init=y_pred_init, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)


print('Total time: %d seconds.' % int(time() - t0))

Clustering stage


C:\Users\bhari\Downloads\EnhancedscDeepCluster\./EnhancedscDeepCluster\model.py:230: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  size_factor = torch.tensor(size_factor, dtype=torch.float64)


Initializing cluster centers with kmeans.
Initializing k-means: NMI= 0.7310, ARI= 0.6351
Clustering   1: NMI= 0.7310, ARI= 0.6351
Epoch   1: Total: 0.36894200 Clustering Loss: 0.14473826 ZINB Loss: 0.22420374
Clustering   2: NMI= 0.7229, ARI= 0.6196
Epoch   2: Total: 0.39492024 Clustering Loss: 0.16923975 ZINB Loss: 0.22568049
Clustering   3: NMI= 0.7249, ARI= 0.6139
Epoch   3: Total: 0.41060636 Clustering Loss: 0.18289070 ZINB Loss: 0.22771567
Clustering   4: NMI= 0.7307, ARI= 0.6224
Epoch   4: Total: 0.41118640 Clustering Loss: 0.18189972 ZINB Loss: 0.22928668
Clustering   5: NMI= 0.7262, ARI= 0.6114
Epoch   5: Total: 0.39167214 Clustering Loss: 0.16158198 ZINB Loss: 0.23009016
Clustering   6: NMI= 0.7331, ARI= 0.6182
Epoch   6: Total: 0.39631587 Clustering Loss: 0.16541630 ZINB Loss: 0.23089957
Clustering   7: NMI= 0.7269, ARI= 0.6094
Epoch   7: Total: 0.37870506 Clustering Loss: 0.14741709 ZINB Loss: 0.23128798
Clustering   8: NMI= 0.7287, ARI= 0.6066
Epoch   8: Total: 0.38204418 C

In [9]:
if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

args.n_clusters = 0

if args.n_clusters > 0:
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X.astype(np.float32), size_factor=adata.obs.size_factors, n_clusters=args.n_clusters, init_centroid=None, 
                y_pred_init=None, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)
else:
    cluster_centers, n_clusters, y_pred_init = model_preprocess.louvain_init_clustering(model=model, adata=adata, knn=args.knn, resolution=args.resolution)
    print('Estimated number of clusters: ', n_clusters)
    y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, n_clusters=n_clusters, init_centroid=cluster_centers, 
                y_pred_init=y_pred_init, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)


print('Total time: %d seconds.' % int(time() - t0))

C:\Users\bhari\Downloads\EnhancedscDeepCluster\project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Estimated number of clusters:  12
Clustering stage


C:\Users\bhari\Downloads\EnhancedscDeepCluster\./EnhancedscDeepCluster\model.py:230: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  size_factor = torch.tensor(size_factor, dtype=torch.float64)


Initializing cluster centers with kmeans.
Initializing k-means: NMI= 0.6696, ARI= 0.4666
Clustering   1: NMI= 0.6923, ARI= 0.4913
Epoch   1: Total: 0.40360253 Clustering Loss: 0.17945546 ZINB Loss: 0.22414707
Clustering   2: NMI= 0.6945, ARI= 0.4920
Epoch   2: Total: 0.45734345 Clustering Loss: 0.23194591 ZINB Loss: 0.22539754
Clustering   3: NMI= 0.6935, ARI= 0.4890
Epoch   3: Total: 0.48689302 Clustering Loss: 0.26006024 ZINB Loss: 0.22683278
Clustering   4: NMI= 0.6929, ARI= 0.4876
Epoch   4: Total: 0.49999684 Clustering Loss: 0.27199246 ZINB Loss: 0.22800438
Clustering   5: NMI= 0.6911, ARI= 0.4860
Epoch   5: Total: 0.47606372 Clustering Loss: 0.24733027 ZINB Loss: 0.22873345
Clustering   6: NMI= 0.6931, ARI= 0.4853
Epoch   6: Total: 0.48568799 Clustering Loss: 0.25619388 ZINB Loss: 0.22949412
Clustering   7: NMI= 0.6914, ARI= 0.4873
Epoch   7: Total: 0.46683215 Clustering Loss: 0.23700514 ZINB Loss: 0.22982701
Clustering   8: NMI= 0.6941, ARI= 0.4841
Epoch   8: Total: 0.44913570 C

Output and save predicted labels and latent features.

In [11]:
if y is not None:
    nmi = np.round(metrics.normalized_mutual_info_score(y, y_pred), 5)
    ari = np.round(metrics.adjusted_rand_score(y, y_pred), 5)
    ss = np.round(metrics.silhouette_score(adata.X, y_pred), 5)
    ch = np.round(metrics.calinski_harabasz_score(adata.X, y_pred), 2)
    print('Evaluating cells:')
    print(f'  NMI = {nmi:.4f}')
    print(f'  ARI = {ari:.4f}')
    print(f'  Silhouette Score = {ss:.4f}')
    print(f'  Calinski-Harabasz Index = {ch:.2f}')

final_latent = model.encodeBatch(torch.tensor(adata.X)).cpu().numpy()
np.savetxt(args.final_latent_file, final_latent, delimiter=",")
np.savetxt(args.predict_label_file, y_pred, delimiter=",", fmt="%i")

Evaluating cells:
  NMI = 0.7256
  ARI = 0.5939
  Silhouette Score = -0.0249
  Calinski-Harabasz Index = 14.36


Run t-SNE on latent features.

In [ ]:
from openTSNE import TSNE

tsne_embedding = TSNE(
                    perplexity=30,
                    initialization="pca",
                    metric="euclidean",
                    n_jobs=8,
                    random_state=42,
                )
latent_tsne_2 = tsne_embedding.fit(final_latent)
np.savetxt("tsne_2D.txt", latent_tsne_2, delimiter=",")

Plot 2D t-SNE of latent features

In [ ]:
rm(list = ls())
library(ggplot2)

latent_tsne <- read.table("tsne_2D.txt", sep=",")
colnames(latent_tsne) <- c("TSNE_1", "TSNE_2")
y_pred <- as.numeric(readLines("pred_labels.txt"))
y_pred <- factor(y_pred, levels=0:max(y_pred))

dat <- data.frame(latent_tsne, y_pred=y_pred)

ggplot(dat, aes(x=TSNE_1, y=TSNE_2, color=y_pred)) + geom_point() + theme_classic()